# Анализ базы данных онлайн приложения

**Цель проекта:** анализ базы данных.

**Задачи исследования:**

1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг — учитывайть только книги с 50 и более оценками;
5. Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных:**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**Этапы исследования:**

1. Подключение к базе данных
2. Предварительный анализ данных
3. Выполнение задач
4. Общий вывод

## Подключение к базе данных

Загрузим нужные библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Установим параметры подключения:

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'], 
                                                         db_config['db'])

Сохраним коннектор:

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Вывод:**

Подключение к базе данных выполнено.

## Предварительный анализ данных

Загрузим первые строки таблиц и изучим их.

Чтобы облегчить работу, создадим функцию, которая будет совершать запрос:

In [4]:
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine)

Теперь выполним запрос по всем табличкам, и сохраним их в переменные для просмотра.

In [5]:
query_1 = '''
SELECT * FROM books
'''
books = select(query_1)

query_2 = '''
SELECT * FROM authors
'''
authors = select(query_2)

query_3 = '''
SELECT * FROM publishers
'''
publishers = select(query_3)

query_4 = '''
SELECT * FROM ratings
'''
ratings = select(query_4)

query_5 = '''
SELECT * FROM reviews
'''
reviews = select(query_5)

Выведем первые строки и краткую информацию о каждой таблице:

In [6]:
tables = [books, authors, publishers, ratings, reviews]
names = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for i in range(len(tables)):
    print('Таблица:', names[i])
    display(tables[i].head(10))
    display(tables[i].info())
    print(' ')

Таблица: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

 
Таблица: authors


...

None

 
Таблица: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

На всякий случай проверим таблицы на наличие дубликатов:

In [7]:
for i in range(len(tables)):
    print('Таблица:', names[i])
    print('Кол-во дубликатов:', tables[i].duplicated().sum())
    print(' ')

Таблица: books
Кол-во дубликатов: 0
 
Таблица: authors
Кол-во дубликатов: 0
 
Таблица: publishers
Кол-во дубликатов: 0
 
Таблица: ratings
Кол-во дубликатов: 0
 
Таблица: reviews
Кол-во дубликатов: 0
 


**Вывод:**

Проблем с таблицами не обнаружено.

## Выполнение задач

### Задача 1

**Посчитать, сколько книг вышло после 1 января 2000 года.**

In [8]:
books_count = '''
SELECT 
    COUNT(book_id) AS books_count
FROM 
    books 
WHERE 
    publication_date > '2000-01-02'
'''

select(books_count)

,books_count
0,819


**Вывод:**

- После 1 января 2000 года вышло 819 книг.

### Задача 2

**Для каждой книги посчитать количество обзоров и среднюю оценку.**

In [9]:
ratings_reviews = '''
SELECT 
    books.title,
    AVG(ratings.rating) AS avg_rating,
    COUNT(DISTINCT reviews.text) AS review_cnt
FROM
    books
LEFT JOIN reviews 
ON reviews.book_id = books.book_id
LEFT JOIN ratings 
ON ratings.book_id = books.book_id
GROUP BY
    books.title
ORDER BY
    review_cnt DESC,
    avg_rating DESC
'''

select(ratings_reviews)

,title,avg_rating,review_cnt
0,Memoirs of a Geisha,4.138462,8
1,Twilight (Twilight #1),3.662500,7
2,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
3,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
4,The Book Thief,4.264151,6
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,4.000000,0
995,Essential Tales and Poems,4.000000,0
996,Leonardo's Notebooks,4.000000,0
997,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0


**Вывод:**

- Самое большое число ревью у книг:
    - Мемуары Гейши: Memoirs of a Geisha
    - Сумерки: Twilight (Twilight #1)
- Самая высокая средняя оценка, в совокупности с самым большим числом ревью у книги: A Dirty Job (Grim Reaper #1)
- Довольно большое число книг со средней оценкой в 5 баллов имеют не больше 2-х ревью.
- Мемуары Гейши в таблице БД встречаются два раза под разными id.

### Задача 3

**Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.**

In [10]:
book_publisher = '''
SELECT 
    publishers.publisher,
    COUNT(books.book_id) AS book_count
FROM 
    publishers
INNER JOIN books 
ON books.publisher_id = publishers.publisher_id
WHERE 
    books.num_pages > 50
GROUP BY 
    publisher
ORDER BY 
    book_count DESC
LIMIT 1
'''

select(book_publisher)

,publisher,book_count
0,Penguin Books,42


**Вывод:**

- Penguin Books - издательство, выпустившее наибольшее число книг (42 шт.) толще 50 страниц.

### Задача 4

**Определить автора с самой высокой средней оценкой книг — учитывайть только книги с 50 и более оценками.**

In [11]:
top_author = '''
WITH
temp AS (SELECT
             authors.author,
             books.book_id AS id,
             AVG(ratings.rating) AS avg_rating,
             COUNT(ratings.rating) AS rating_count
         FROM
             authors
         INNER JOIN books 
         ON books.author_id = authors.author_id
         INNER JOIN ratings 
         ON ratings.book_id = books.book_id
         GROUP BY
             author,
             id
         HAVING
             COUNT(ratings.rating) >= 50)
         
SELECT
    temp.author,
    AVG(temp.avg_rating) AS avg_rating
FROM
    temp
GROUP BY
    author
ORDER BY
    avg_rating DESC
LIMIT 1
'''

select(top_author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


**Вывод:**

- Автор с самой высокой средней оценкой книг (от 50 оценок и более): J.K. Rowling/Mary GrandPré

### Задача 5

**Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [12]:
avg_review_count = '''
WITH
temp1 AS (SELECT 
              username,
          COUNT(rating_id) AS count
          FROM 
              ratings
          GROUP BY
              username),
              
temp2 AS (SELECT
              COUNT(text)
          FROM 
              reviews
          FULL OUTER JOIN temp1 
          ON temp1.username = reviews.username
          WHERE 
              temp1.count > 50
          GROUP BY 
              reviews.username)
              
SELECT 
    ROUND(avg(count)) AS avg_review_count
FROM
    temp2
'''

select(avg_review_count)

,avg_review_count
0,24.0


**Вывод:**

- Среднее кол-во обзоров от пользователей, которые поставили больше 50 оценок, равно 24.

## Общий вывод

Мы проверили данные, и выполнили поставленные задачи. 

Как итог, мы постановили следующее:
- После 1 января 2000 года вышло 819 книг.
- Самое большое число ревью у книг:
    - Мемуары Гейши: Memoirs of a Geisha
    - Сумерки: Twilight (Twilight #1)
- Самая высокая средняя оценка, в совокупности с самым большим числом ревью у книги: A Dirty Job (Grim Reaper #1)
- Довольно большое число книг со средней оценкой в 5 баллов имеют не больше 2-х ревью.
- Мемуары Гейши в таблице БД встречаются два раза под разными id.
- Penguin Books - издательство, выпустившее наибольшее число книг (42 шт.) толще 50 страниц.
- Среднее кол-во обзоров от пользователей, которые поставили больше 50 оценок, равно 24.